<a href="https://colab.research.google.com/github/Vakhranev/Compling/blob/master/HW02_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Реализуем триграмную модель

In [0]:
from google.colab import files
uploaded = files.upload()

Saving anna_karenina.txt to anna_karenina.txt
Saving besy_dostoevsky.txt to besy_dostoevsky.txt


In [0]:
dostoevsky = open('besy_dostoevsky.txt').read()
tolstoy = open('anna_karenina.txt').read()

In [247]:
print("Длина Бесов Достоевского -", len(dostoevsky))
print("Длина Анны Карениной Толстого - ", len(tolstoy))

Длина Бесов Достоевского - 1293557
Длина Анны Карениной Толстого -  1710408


In [0]:
from string import punctuation
import numpy as np

def normalize(text):
    normalized_text = [word.strip(punctuation) for word \
                                                            in text.lower().split()]
    normalized_text = [word for word in normalized_text if word]
    return normalized_text

In [0]:
norm_dostoevsky = normalize(dostoevsky)
norm_tolstoy = normalize(tolstoy)

In [250]:
print("Длина Бесов Достоевского в токенах -", len(norm_dostoevsky))
print("Длина Анны Карениной Толстого в токенах - ", len(norm_tolstoy))

Длина Бесов Достоевского в токенах - 208453
Длина Анны Карениной Толстого в токенах -  281201


In [0]:
from collections import Counter
from nltk.tokenize import sent_tokenize
def ngrammer(tokens, n=2):
    ngrams = []
    for i in range(0,len(tokens)-n+1):
        ngrams.append(' '.join(tokens[i:i+n]))
    return ngrams

In [252]:
import nltk
nltk.download('punkt')
sentences_dostoevsky = [['<start>'] + ['<start>'] + normalize(text) + ['<end>'] for text in sent_tokenize(dostoevsky)]
sentences_tolstoy = [['<start>'] + ['<start>'] + normalize(text) + ['<end>'] for text in sent_tokenize(tolstoy)]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [0]:
unigrams_dostoevsky = Counter()
bigrams_dostoevsky = Counter()

dut = sentences_dostoevsky[:1000]
for sentence in dut:
    unigrams_dostoevsky.update(sentence)
    bigrams_dostoevsky.update(ngrammer(sentence))

unigrams_tolstoy = Counter()
bigrams_tolstoy = Counter()

cut = sentences_tolstoy[:1000]
for sentence in cut:
    unigrams_tolstoy.update(sentence)
    bigrams_tolstoy.update(ngrammer(sentence))

In [0]:
from collections import Counter
unigrams_dostoevsky = Counter()
trigrams_dostoevsky = Counter()

for sentence in dut:
    unigrams_dostoevsky.update(sentence)
    trigrams_dostoevsky.update(ngrammer(sentence, n=3))
    
unigrams_tolstoy = Counter()
trigrams_tolstoy = Counter()

for sentence in cut:
    unigrams_tolstoy.update(sentence)
    trigrams_tolstoy.update(ngrammer(sentence, n=3))

In [255]:
print(len(unigrams_dostoevsky))

6115


In [256]:
print(trigrams_tolstoy.most_common(10))

[('<start> <start> –', 393), ('<start> <start> он', 41), ('<start> <start> и', 36), ('– сказал он', 32), ('<start> <start> степан', 30), ('<start> степан аркадьич', 30), ('<start> <start> но', 28), ('<start> – ну', 28), ('<start> <start> она', 27), ('<start> <start> левин', 26)]


In [0]:
matrix_tolstoy = np.zeros((len(unigrams_tolstoy), 
                   len(unigrams_tolstoy)))

id2word_unigrams = list(unigrams_tolstoy)
word2id_unigrams = {word:i for i, word in enumerate(id2word_unigrams)}
id2word_bigrams = list(bigrams_tolstoy)
word2id_bigrams = {coll:d for d, coll in enumerate(id2word_bigrams)}

for ngram in bigrams_tolstoy:
    word1, word2 = ngram.split()
    matrix_tolstoy[word2id_unigrams[word1]][word2id_unigrams[word2]] =  (bigrams_tolstoy[ngram]/
                                                                     unigrams_tolstoy[word1])

In [0]:
matrix_tolstoy = np.zeros((len(bigrams_tolstoy), 
                   len(unigrams_tolstoy)))

for ngram in trigrams_tolstoy:
    word1, word2, word3 = ngram.split()
    bigram = '{} {}'.format(word1, word2)
    matrix_tolstoy[word2id_bigrams[bigram]][word2id_unigrams[word3]] =  (trigrams_tolstoy[ngram]/
                                                                     bigrams_tolstoy[bigram])

In [0]:
def generate(matrix, id2word, word2id, n=100, start=['<start>','<start>']):
    text = []
    current_bigram = start
    current_idx = word2id[' '.join(current_bigram)]
    
    for i in range(n):  
        chosen = np.random.choice(matrix.shape[1], p=matrix[current_idx])
        
        text.append(id2word[chosen])
        
        if id2word[chosen] == '<end>':
            current_bigram = start
            current_idx = word2id[' '.join(current_bigram)]
        else:
          current_bigram = [current_bigram[1], id2word[chosen]]
          current_idx = word2id[' '.join(current_bigram)]


    
    return ' '.join(text)

In [260]:
print(generate(matrix_tolstoy, id2word_unigrams, word2id_bigrams).replace('<end>', '\n'))

знакомый встретился и окликнул его но левин был его товарищем и другом первой молодости что чувство это не любовь 
 – нет уж не может же это все стесняет – сказал сергей иваныч покачивая головой 
 я огорчил ее 
 авось бог даст 
 но только дикость моя не в том что ты 
 это от вас зависит – сказал он с значительным видом склонив голову набок уставился на барина 
 во время своего студенчества он чуть было не строго глаза смотрели так же как и почему он спит не в силах удерживать улыбки говорил левин 
 – эге 



In [0]:
matrix_dostoevsky = np.zeros((len(unigrams_dostoevsky), 
                   len(unigrams_dostoevsky)))

dosid2word_unigrams = list(unigrams_dostoevsky)
dosword2id_unigrams = {word:i for i, word in enumerate(dosid2word_unigrams)}
dosid2word_bigrams = list(bigrams_dostoevsky)
dosword2id_bigrams = {coll:d for d, coll in enumerate(dosid2word_bigrams)}

for mgram in bigrams_dostoevsky:
    words1, words2 = mgram.split()
    matrix_dostoevsky[dosword2id_unigrams[words1]][dosword2id_unigrams[words2]] =  (bigrams_dostoevsky[mgram]/
                                                                     unigrams_dostoevsky[words1])

In [0]:
matrix_dostoevsky = np.zeros((len(bigrams_dostoevsky), 
                   len(unigrams_dostoevsky)))

for mgram in trigrams_dostoevsky:
    words1, words2, words3 = mgram.split()
    dosbigram = '{} {}'.format(words1, words2)
    matrix_dostoevsky[dosword2id_bigrams[dosbigram]][dosword2id_unigrams[words3]] =  (trigrams_dostoevsky[mgram]/
                                                                     bigrams_dostoevsky[dosbigram])

In [263]:
print(generate(matrix_dostoevsky, dosid2word_unigrams, dosword2id_bigrams).replace('<end>', '\n'))

прибавляли сверх того по другим отчасти уже деликатным причинам 
 шатову очень хотелось ему тогда за значительный карточный проигрыш внезапно обнаружившийся 
 тогда еще не успели ничего понять да им и не думая о ней надо бы поговорить особенно что я развратитель молодежи и рассадник губернского атеизма… он тотчас же по возвращении вознаградить тем более что постоянно проигрывал 
 что может быть и правы вы ведь читали сейчас письмо дроздовой что могло быть удовлетворено 
 она конечно понимала что ей нельзя водиться с этими людьми но все-таки принимала их с жадностию прочла это первое письмо и подчеркнув карандашом восклицание «где


Тексты, сгенерированные при помощи триграммов, больше похожи на человеческую речь, чем тексты, созданные биграммами, так как выбор следующего слова происходит исходя из большего контекста.

2. Gensim.models.Phrases

In [264]:
import re, collections
import nltk
nltk.download('punkt')
from nltk import sent_tokenize
from nltk import word_tokenize
from gensim.models.phrases import Phrases, Phraser
from gensim.models.word2vec import Text8Corpus
from gensim.test.utils import datapath
 
text="""В конце 1811 г., в эпоху нам достопамятную, жил в своем поместье Ненарадове добрый Гаврила Гаврилович Р**. Он славился во всей округе гостеприимством и радушием; соседи поминутно ездили к нему поесть, попить, поиграть по пяти копеек в бостон с его женою, а некоторые для того, чтоб поглядеть на дочку их, Марью Гавриловну, стройную, бледную и семнадцатилетнюю девицу. Она считалась богатой невестою, и многие прочили ее за себя или за сыновей. Марья Гавриловна была воспитана на французских романах, и следственно была влюблена. Предмет, избранный ею, был бедный армейский прапорщик, находившийся в отпуску в своей деревне."""
sentences = sent_tokenize(text)


for sents in sentences:
    print(sents)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
В конце 1811 г., в эпоху нам достопамятную, жил в своем поместье Ненарадове добрый Гаврила Гаврилович Р**.
Он славился во всей округе гостеприимством и радушием; соседи поминутно ездили к нему поесть, попить, поиграть по пяти копеек в бостон с его женою, а некоторые для того, чтоб поглядеть на дочку их, Марью Гавриловну, стройную, бледную и семнадцатилетнюю девицу.
Она считалась богатой невестою, и многие прочили ее за себя или за сыновей.
Марья Гавриловна была воспитана на французских романах, и следственно была влюблена.
Предмет, избранный ею, был бедный армейский прапорщик, находившийся в отпуску в своей деревне.


In [266]:
def split(word): 
    return [char for char in word] 
listik = []
for i in range(len(sentences)):
    listik.append(split(sentences[i]))
print(listik)

[['В', ' ', 'к', 'о', 'н', 'ц', 'е', ' ', '1', '8', '1', '1', ' ', 'г', '.', ',', ' ', 'в', ' ', 'э', 'п', 'о', 'х', 'у', ' ', 'н', 'а', 'м', ' ', 'д', 'о', 'с', 'т', 'о', 'п', 'а', 'м', 'я', 'т', 'н', 'у', 'ю', ',', ' ', 'ж', 'и', 'л', ' ', 'в', ' ', 'с', 'в', 'о', 'е', 'м', ' ', 'п', 'о', 'м', 'е', 'с', 'т', 'ь', 'е', ' ', 'Н', 'е', 'н', 'а', 'р', 'а', 'д', 'о', 'в', 'е', ' ', 'д', 'о', 'б', 'р', 'ы', 'й', ' ', 'Г', 'а', 'в', 'р', 'и', 'л', 'а', ' ', 'Г', 'а', 'в', 'р', 'и', 'л', 'о', 'в', 'и', 'ч', ' ', 'Р', '*', '*', '.'], ['О', 'н', ' ', 'с', 'л', 'а', 'в', 'и', 'л', 'с', 'я', ' ', 'в', 'о', ' ', 'в', 'с', 'е', 'й', ' ', 'о', 'к', 'р', 'у', 'г', 'е', ' ', 'г', 'о', 'с', 'т', 'е', 'п', 'р', 'и', 'и', 'м', 'с', 'т', 'в', 'о', 'м', ' ', 'и', ' ', 'р', 'а', 'д', 'у', 'ш', 'и', 'е', 'м', ';', ' ', 'с', 'о', 'с', 'е', 'д', 'и', ' ', 'п', 'о', 'м', 'и', 'н', 'у', 'т', 'н', 'о', ' ', 'е', 'з', 'д', 'и', 'л', 'и', ' ', 'к', ' ', 'н', 'е', 'м', 'у', ' ', 'п', 'о', 'е', 'с', 'т', 'ь', ',', '

In [267]:
phrases = Phrases(listik, min_count=15, scoring='npmi', threshold=-1)
p1 = Phrases (phrases[listik], min_count=10, scoring='npmi', threshold=-1)
p2 = Phrases (p1[phrases[listik]], min_count=1, scoring='npmi', threshold=0)
p3 = Phrases (p2[p1[phrases[listik]]], min_count=2, scoring='npmi', threshold=0.1)
print(list(p3[p2[p1[listik]]]))

[['В_ ', 'к_о', 'н_ц', 'е_ ', '1_8', '1_1', ' _г', '._,_ ', 'в_ ', 'э_п', 'о_х', 'у_ ', 'н_а_м', ' _д', 'о_с_т_о', 'п_а', 'м_я', 'т_н', 'у_ю', ',_ _ж', 'и_л_ ', 'в_ _с', 'в_о', 'е_м', ' _п_о', 'м_е', 'с_т', 'ь_е', ' _Н', 'е_н_а', 'р_а', 'д_о', 'в_е', ' _д', 'о_б', 'р_ы', 'й_ ', 'Г_а_в_р', 'и_л_а_ ', 'Г_а_в_р', 'и_л_о', 'в_и', 'ч_ ', 'Р_*', '*_.'], ['О_н', ' _с_л', 'а_в', 'и_л_с', 'я_ ', 'в_о', ' _в', 'с_е', 'й_ ', 'о_к', 'р_у', 'г_е', ' _г', 'о_с', 'т', 'е_п', 'р', 'и', 'и_м', 'с_т', 'в_о', 'м_ ', 'и_ ', 'р_а', 'д_у', 'ш_и', 'е_м', ';_ _с', 'о_с', 'е_д', 'и_ _п', 'о_м', 'и', 'н_у', 'т_н', 'о', ' ', 'е_з', 'д_и_л', 'и_ ', 'к_ ', 'н_е', 'м_у', ' _п_о', 'е_с', 'т_ь', ',_ _п', 'о_п', 'и_т', 'ь_,_ ', 'п_о', 'и_г', 'р_а', 'т_ь', ' _п_о', ' _п_я', 'т', 'и_ ', 'к_о', 'п_е', 'е_к', ' _в', ' _б', 'о_с_т_о', 'н_ _с', ' ', 'е_г', 'о', ' _ж', 'е_н', 'о_ю', ',_ _а_ ', 'н_е', 'к_о', 'т_о', 'р_ы', 'е_ ', 'д_л', 'я_ ', 'т_о', 'г_о', ',_ _ч', 'т_о', 'б_ _п', 'о_г', 'л_я', 'д_е', 'т_ь', ' _н_а', ' _д', '

/usr/local/lib/python3.6/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [268]:
letters = phrases[listik]
for i in range(100):
    phrases = Phrases(letters, min_count=1, scoring='npmi', threshold=-1)
    letters = phrases[letters]
print(list(phrases[letters]))

/usr/local/lib/python3.6/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


[['В_ _к_о_н_ц_е_ _1_8_1_1_ _г_._,_ _в_ _э_п_о_х_у_ _н_а_м_ _д_о_с_т_о_п_а_м_я_т_н_у_ю_,_ _ж_и_л_ _в_ _с_в_о_е_м_ _п_о_м_е_с_т_ь_е_ _Н_е_н_а_р_а_д_о_в_е_ _д_о_б_р_ы_й_ _Г_а_в_р_и_л_а_ _Г_а_в_р_и_л_о_в_и_ч_ _Р_*_*_.'], ['О_н_ _с_л_а_в_и_л_с_я_ _в_о_ _в_с_е_й_ _о_к_р_у_г_е_ _г_о_с_т_е_п_р_и_и_м_с_т_в_о_м_ _и_ _р_а_д_у_ш_и_е_м_;_ _с_о_с_е_д_и_ _п_о_м_и_н_у_т_н_о_ _е_з_д_и_л_и_ _к_ _н_е_м_у_ _п_о_е_с_т_ь_,_ _п_о_п_и_т_ь_,_ _п_о_и_г_р_а_т_ь_ _п_о_ _п_я_т_и_ _к_о_п_е_е_к_ _в_ _б_о_с_т_о_н_ _с_ _е_г_о_ _ж_е_н_о_ю_,_ _а_ _н_е_к_о_т_о_р_ы_е_ _д_л_я_ _т_о_г_о_,_ _ч_т_о_б_ _п_о_г_л_я_д_е_т_ь_ _н_а_ _д_о_ч_к_у_ _и_х_,_ _М_а_р_ь_ю_ _Г_а_в_р_и_л_о_в_н_у_,_ _с_т_р_о_й_н_у_ю_,_ _б_л_е_д_н_у_ю_ _и_ _с_е_м_н_а_д_ц_а_т_и_л_е_т_н_ю_ю_ _д_е_в_и_ц_у_.'], ['О_н_а_ _с_ч_и_т_а_л_а_с_ь_ _б_о_г_а_т_о_й_ _н_е_в_е_с_т_о_ю_,_ _и_ _м_н_о_г_и_е_ _п_р_о_ч_и_л_и_ _е_е_ _з_а_ _с_е_б_я_ _и_л_и_ _з_а_ _с_ы_н_о_в_е_й_.'], ['М_а_р_ь_я_ _Г_а_в_р_и_л_о_в_н_а_ _б_ы_л_а_ _в_о_с_п_и_т_а_н_а_ _н_а_ _ф_р_а_н_ц_у_з_с_к_и_х_ _р_о_м_